In [1]:
from ga_regression import *
from matplotlib import pyplot as plt
from functools import partial
import PIL
import matplotlib as mpl
from matplotlib.cm import ScalarMappable
from pvutils import iter_subplots
from pyvista import PolyData
import pyvista as pv
from scipy.stats import pearsonr

C:\Users\dg\.conda\envs\diffnet4\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
file = Path(r"D:\resynth\run_48_49\many_faces\2025-07-14-17-10-07\opts_and_metadata.pt")
d = torch.load(file)
metas = d['metadata']
opts = d['opts']
train_scenes, test_scenes = d['train_scenes'], d['test_scenes']

In [5]:
readers = self = Readers([m.reader() for m in metas])

In [12]:
readers.hparams.dtypes

log_folder         object
model_file         object
metadata_file      object
input_features     object
channel            object
k_eig               int64
learning_rate     float64
decay_every         int64
decay_rate        float64
dropout              bool
n_faces             int64
spike_window       object
weight_error       object
dtype: object

In [15]:
exclude = ()
tags = [
    k for k, v in self.hparams.items()
    if len(set(v)) > 1 and k not in exclude and v.dtype != 'object'
]
tags

['n_faces']

In [13]:
readers.hparams

,log_folder,model_file,metadata_file,input_features,channel,k_eig,learning_rate,decay_every,decay_rate,dropout,n_faces,spike_window,weight_error
0,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,hks,"(14, 17, 29, 23, 2, 0, 13, 31, 3, 26, 28, 9, 2...",128,0.000063,50,0.5,True,1500,"(0.07, 0.75)",response
1,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,hks,"(14, 17, 29, 23, 2, 0, 13, 31, 3, 26, 28, 9, 2...",128,0.000063,50,0.5,True,1250,"(0.07, 0.75)",response
2,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,hks,"(14, 17, 29, 23, 2, 0, 13, 31, 3, 26, 28, 9, 2...",128,0.000063,50,0.5,True,1000,"(0.07, 0.75)",response
3,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,hks,"(14, 17, 29, 23, 2, 0, 13, 31, 3, 26, 28, 9, 2...",128,0.000063,50,0.5,True,750,"(0.07, 0.75)",response
4,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,D:\resynth\run_48_49\many_faces\2025-07-14-17-...,hks,"(14, 17, 29, 23, 2, 0, 13, 31, 3, 26, 28, 9, 2...",128,0.000063,50,0.5,True,500,"(0.07, 0.75)",response


In [28]:
# TODO Scatter plotts

True